# 多线程与线程池
## 多线程
###### •	特点：使用多个线程同时执行任务，适用于 CPU 计算任务或 I/O 密集型任务。
###### •    缺点：Python 的 GIL（全局解释器锁）限制了 Python 线程的真正并行执行，但对 I/O 操作（如爬虫）仍然有明显加速效果。

# 比较单线程和多线程所用时间

In [6]:
import requests
import time

urls = 'https://www.baidu.com','https://www.sogou.com','https://www.sohu.com'

start_time = time.time()

for url in urls:
    req=requests.get(url)
    time.sleep(2)


start_end = time.time()
print('花费时间:',start_end - start_time,'s')




花费时间: 6.252176284790039


In [9]:
import  threading

urls = 'https://www.baidu.com','https://www.sogou.com','https://www.sohu.com'

start_time = time.time()

def func1(url):
    response=requests.get(url)
    time.sleep(2)

start_end = time.time()

threads=[]
for url in urls:
    thread=threading.Thread(target=func1,args=(url,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

start_end = time.time()
print('花费时间:',start_end - start_time,'s')

花费时间: 2.1530039310455322


# 线程池
###### multiprocessing： 适合 CPU 密集型任务，如计算密集的操作。
###### multiprocessing.dummy： 适合 I/O 密集型任务，如文件读写、网络请求等。

In [27]:
from multiprocessing.dummy import Pool
import time


urls = 'https://www.baidu.com','https://www.sogou.com','https://www.sohu.com'

start_time = time.time()

def func1(url):
    response=requests.get(url)
    time.sleep(2)

pool = Pool(3)
pool.map(func1,urls)
pool.close()
pool.join()

start_end = time.time()
print('花费时间:',start_end - start_time)

花费时间: 2.1556339263916016


##### 在使用from multiprocessing import Pool经常会报错：AttributeError: Can't get attribute 'func1' on <module '__main__' (built-in)>
##### 运行多进程程序时出现错误AttributeError: Can't get attribute 'long_time_task' on <module '__main__' (built-in)>。由于某些原因Pool不能使用未在导入的模块中定义的对象。解决的方法是你必须将函数写入不同的文件模块并导入该模块。
##### 解决办法：一个文件产生错误，把函数定义放在另一个py文件中再引入。
##### 这个问题似乎是多处理器的一个设计缺陷，具体请参阅：https://bugs.python.org/issue25053

# 使用进程池爬取红楼梦小说

In [36]:
from lxml import etree
import os
import time
from curl_cffi import requests

ch_url='https://m.shicimingju.com/book/hongloumeng.html'

headers={'use-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}

if not os.path.exists('./data'):
    os.makedirs('./data')

fp = open('./data/hongloumeng.txt','w',encoding='utf-8')

start_time = time.time()

title_res=requests.get(ch_url, headers=headers)
title_res.encoding = "utf-8"
ch_tree=etree.HTML(title_res.text)
ch_list=ch_tree.xpath('/html/body/div[3]/div[2]/div[3]/div//a')

# print(ch_list)

for a in ch_list:
    #获取tilte
    title=a.text.strip()
    #获取内容
    content_url="https://m.shicimingju.com" + a.get('href')

    print('正在爬取：',title,'--',content_url)

    content_res=requests.get(content_url, headers=headers)
    content_res.encoding = "utf-8"
    tree=etree.HTML(content_res.text)
    content=tree.xpath('/html/body/div[3]/div[2]/div[2]/div//text()')

    content='\n'.join(content)
    fp.write(title+':'+'\n'+ content +'\n')

print('爬取完成！')

end_time = time.time()

print(f'花费时间:{end_time - start_time:.2f}s')

正在爬取： 第 一 回 甄士隐梦幻识通灵 贾雨村风尘怀闺秀 -- https://m.shicimingju.com/book/hongloumeng/1.html
正在爬取： 第 二 回 贾夫人仙逝扬州城 冷子兴演说荣国府 -- https://m.shicimingju.com/book/hongloumeng/2.html
正在爬取： 第 三 回 托内兄如海酬训教 接外孙贾母惜孤女 -- https://m.shicimingju.com/book/hongloumeng/3.html
正在爬取： 第 四 回 薄命女偏逢薄命郎 葫芦僧乱判葫芦案 -- https://m.shicimingju.com/book/hongloumeng/4.html
正在爬取： 第 五 回 游幻境指迷十二钗 饮仙醪曲演红楼梦 -- https://m.shicimingju.com/book/hongloumeng/5.html
正在爬取： 第 六 回 贾宝玉初试云雨情 刘姥姥一进荣国府 -- https://m.shicimingju.com/book/hongloumeng/6.html
正在爬取： 第 七 回 送宫花贾琏戏熙凤 宴宁府宝玉会秦钟 -- https://m.shicimingju.com/book/hongloumeng/7.html
正在爬取： 第 八 回 比通灵金莺微露意 探宝钗黛玉半含酸 -- https://m.shicimingju.com/book/hongloumeng/8.html
正在爬取： 第 九 回 恋风流情友入家塾 起嫌疑顽童闹学堂 -- https://m.shicimingju.com/book/hongloumeng/9.html
正在爬取： 第 十 回 金寡妇贪利权受辱 张太医论病细穷源 -- https://m.shicimingju.com/book/hongloumeng/10.html
正在爬取： 第十一回 庆寿辰宁府排家宴 见熙凤贾瑞起淫心 -- https://m.shicimingju.com/book/hongloumeng/11.html
正在爬取： 第十二回 王熙凤毒设相思局 贾天祥正照 -- https://m.shicimingju.com/book/hongloumeng/12.html
正在爬取： 

# 使用进程池

In [37]:
from lxml import etree
import os
import time
from curl_cffi import requests

ch_url='https://m.shicimingju.com/book/hongloumeng.html'

headers={'use-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}

if not os.path.exists('./data'):
    os.makedirs('./data')

fp = open('./data/hongloumeng.txt','w',encoding='utf-8')

start_time = time.time()


def func1(ch_url):
    title_res=requests.get(ch_url, headers=headers)
    title_res.encoding = "utf-8"
    ch_tree=etree.HTML(title_res.text)
    ch_list=ch_tree.xpath('/html/body/div[3]/div[2]/div[3]/div//a')
    data=[]
    for a in ch_list:
        #获取tilte
        title=a.text.strip()
        #获取内容
        content_url="https://m.shicimingju.com" + a.get('href')
        data.append((title,content_url))#append() 只能接受 一个 参数，而 (title, content_url) 是 一个元组，所以它是正确的写法
    return data



def func2(data):
    title,url=data
    content_res=requests.get(url, headers=headers)
    content_res.encoding = "utf-8"
    tree=etree.HTML(content_res.text)
    content=tree.xpath('/html/body/div[3]/div[2]/div[2]/div//text()')
    print('正在爬取：',title,'--',url)
    content='\n'.join(content)
    fp.write(title+':'+'\n'+ content +'\n')


data=func1(ch_url)

pool = Pool(4)
pool.map(func2,data)
pool.close()
pool.join()

print('爬取完成！')

end_time = time.time()

print(f'花费时间:{end_time - start_time:.2f}s')

正在爬取： 第二十五回 魇魔法叔嫂逢五鬼 红楼梦通灵遇双真 -- https://m.shicimingju.com/book/hongloumeng/25.html
正在爬取： 第 九 回 恋风流情友入家塾 起嫌疑顽童闹学堂 -- https://m.shicimingju.com/book/hongloumeng/9.html
正在爬取： 第十七回 大观园试才题对额 荣国府归省庆元宵 -- https://m.shicimingju.com/book/hongloumeng/17.html
正在爬取： 第 一 回 甄士隐梦幻识通灵 贾雨村风尘怀闺秀 -- https://m.shicimingju.com/book/hongloumeng/1.html
正在爬取： 第二十六回 蜂腰桥设言传密意 馆春困发幽情 -- https://m.shicimingju.com/book/hongloumeng/26.html
正在爬取： 第 十 回 金寡妇贪利权受辱 张太医论病细穷源 -- https://m.shicimingju.com/book/hongloumeng/10.html
正在爬取： 第 二 回 贾夫人仙逝扬州城 冷子兴演说荣国府 -- https://m.shicimingju.com/book/hongloumeng/2.html
正在爬取： 第十八回 隔珠帘父女勉忠勤 搦湘管姊弟裁题咏 -- https://m.shicimingju.com/book/hongloumeng/18.html
正在爬取： 第十一回 庆寿辰宁府排家宴 见熙凤贾瑞起淫心 -- https://m.shicimingju.com/book/hongloumeng/11.html
正在爬取： 第二十七回 滴翠亭杨妃戏彩蝶 埋香冢飞燕泣残红 -- https://m.shicimingju.com/book/hongloumeng/27.html
正在爬取： 第十九回 情切切良宵花解语 意绵绵静日玉生香 -- https://m.shicimingju.com/book/hongloumeng/19.html
正在爬取： 第 三 回 托内兄如海酬训教 接外孙贾母惜孤女 -- https://m.shicimingju.com/book/hongloumeng/3.html
正在

#### 单线程使用时间：花费时间:19.08s
#### 线程池使用时间：花费时间:5.69s